# Random Forest Baseline Advanced Optimization 

Cloned from: [Feature_selection_LR_airlinejoin (baseline-3)](tobe update)
- Has the hyperparameter tuning for the baseline with all features for midterm.

- What is this notebook for? 
  - Notebook that runs the baseline Random Forest with advanced optimization
  - Machine Learning Algorithms : Random Forest
  - Metrics: F1, accuracy, precision
  
---
  
- Data:
  - Final, cleaned weather
  - Cleaned flights minus (PR departure delay, PR arrival delay, carrier ordinal)

RF in spark

https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1898361324232092/command/1898361324232093

Feature selection in RF

https://antonhaugen.medium.com/feature-selection-for-pyspark-tree-classifiers-3be54a8bc493

# 0. Preparation

In [0]:
import pandas as pd
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import ast
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import geopandas as gpd
import folium
import time
import hyperopt

from pyspark.sql.functions import col, max, min
from pyspark.sql.functions import col,isnan,when,count
from pyspark.sql.functions import desc
from pyspark.sql.types import DateType
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import MinMaxScaler
from pyspark.mllib.evaluation import MulticlassMetrics, BinaryClassificationMetrics
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.functions import add_months, date_add
from pyspark.ml.classification import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

from hyperopt import tpe, hp, fmin, STATUS_OK
from hyperopt.pyll.base import scope

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from hyperopt import Trials



In [0]:
# Put at the top of any notebooks for storing in blob

from pyspark.sql.functions import col, max

blob_container = "team06" # The name of your container created in https://portal.azure.com
storage_account = "apatel" # The name of your Storage account created in https://portal.azure.com
secret_scope = "team06" # The name of the scope created in your local computer using the Databricks CLI
secret_key = "team06" # The name of the secret key created in your local computer using the Databricks CLI 
blob_url = f"wasbs://{blob_container}@{storage_account}.blob.core.windows.net"
mount_path = "/mnt/mids-w261"

#Matt RF Manual Simplified

This is a streamlined version of the original tuning notebook to more quickly tune a model for each fold for use in ensembling/voting.

We export both the validation and test set predictions, as the validation set will be used to tune the threshold.

In [0]:
def rebalanceDF(trainingDF, desired_neg_to_pos_ratio):
  original_dep_del15_count_df = trainingDF.groupby('DEP_DEL15').count()
  positiveTrainCount = original_dep_del15_count_df.filter(original_dep_del15_count_df.DEP_DEL15 ==  1).select(['count']).head()[0]
  negativeTrainCount = original_dep_del15_count_df.filter(original_dep_del15_count_df.DEP_DEL15 ==  0).select(['count']).head()[0]

  # undersampling the negative cases
  negativeDF = trainingDF.filter(trainingDF['DEP_DEL15']==0).sample(False, positiveTrainCount/negativeTrainCount*desired_neg_to_pos_ratio, seed=12345)
  positiveDF = trainingDF.filter(trainingDF['DEP_DEL15']==1)
  new_trainDF = positiveDF.union(negativeDF).cache()
  return new_trainDF

In [0]:
df_imported = spark.read.parquet(f'{blob_url}/full_join_mattsFeats_anandFeats_cleaned_extraFeatures_upToOD_good_v8_4-6-22')

In [0]:
feature_list_full = ['DISTANCE', 'ORI_elevation', 'DEST_elevation', 'prior_dep_delayed', 'previous_DEP_DELAY_NEW', 'prior_arr_delayed', 'previous_ARR_DELAY_NEW', 'plane_is_here', 'avg_carrier_delay_24hrs', 'flights_sch_Today_ORIGIN', 'flights_sch_Today_DEST', 'avg_ori_airport_delay_24hrs', 'daytime', 'evening', 'new_england', 'mid_atlantic', 'south', 'midwest', 'southwest', 'west', 'pacific_islands', 'spring', 'winter', 'autumn', 'weekend_or_holiday', 'prior_dep_delayed', 'previous_DEP_DELAY_NEW', 'plane_is_here', 'previous_ARR_DELAY_NEW', 'avg_carrier_delay_24hrs', 'prior_arr_delayed', 'avg_ori_airport_delay_24hrs', 'depDelayPageRank', 'arrDelayPageRank', 'OC1_0_wind_gust_spd_rate_imp', 'AA1_1_liquid_precip', 'AA3_1_liquid_precip', 'gd1_0_sky_coverage', 'au2_4_extreme_wind_weather', 'mv1_0_sand_dust_near', 'mv1_0_thunder_rain_near', 'aw1_mw1_0_smoke_haze_dust', 'aw1_mw1_0_fog', 'aw1_mw1_0_rain_drizzle', 'aw1_mw1_0_freezing_rain_drizzle', 'aw1_mw1_0_snow', 'aw1_mw1_0_hail_or_ice', 'aw1_mw1_0_thunderstorm', 'aw1_mw1_0_tornado', 'SLP_0_avg_station_press_imp', 'WND_3_wind_speed_imp', 'CIG_0_sky_ceiling_height_imp', 'VIS_0_visibility_dist_imp', 'TMP_0_air_temperature_imp', 'DEW_0_dew_pt_temp_imp', 'MA1_0_altimeter_set_rate_imp', 'MA1_2_station_pres_rate_imp', 'GD1_3_cloud_height_imp', 'avg_hourly_delay_24hr', 'avg_OD_dep_del15_24hr', 'avg_OD_delay_min_24hr', 'avg_OD_num_flights_24hr', 'depDelayPageRank_ordinal', 'arrDelayPageRank_ordinal', 'departed_for_current_aiport', 'time_inb_flight_min', 'avg_time_inb_flights_carrier_24hr', 'avg_time_inb_flights_origin_24hr', 'avg_time_inb_flights_dest_24hr', 'avg_ori_DEP_DELAY_NEW_24hr', 'airline_carrier_del', 'airline_carrier_del_min', 'avg_carrier_delay_over15_lastQ', 'airline_carrier_del_ordinal', 'airline_carrier_del_min_ordinal', 'origin_avg_DEP_DEL15', 'origin_avg_DEP_DELAY_NEW', 'avg_origin_delay_over15_lastQ', 'origin_del_ordinal', 'origin_del_min_ordinal', 'OD_avg_DEP_DEL15', 'OD_avg_DEP_DELAY_NEW', 'avg_OD_delay_over15_lastQ', 'OD_del_ordinal', 'OD_del_min_ordinal']

In [0]:
desired_neg_to_pos_ratio = 1.6

maxDepth=8
maxBins=64
minInstancesPerNode=10
numTrees=100
seed=12345

CV_name = 'fold_1'
train_data = df_imported.where((col('YEAR').cast('int') == 2015))
val_data = df_imported.where((col('YEAR').cast('int') == 2016))
holdout_data = df_imported.where((col('YEAR').cast('int') == 2019))
save_name = 'xgboost_test_preds_4_9_'+CV_name

new_trainDF = rebalanceDF(train_data, desired_neg_to_pos_ratio)
val = val_data
holdout = holdout_data

# Assemble features
assembler = VectorAssembler().setInputCols(feature_list_full).setOutputCol('features')
assmb_train = assembler.transform(new_trainDF)
assmb_val = assembler.transform(val)
assmb_holdout = assembler.transform(holdout)

# modeling
rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'DEP_DEL15',
                            maxDepth=maxDepth, maxBins=maxBins, minInstancesPerNode=minInstancesPerNode, numTrees=numTrees, seed=seed)
rfModel = rf.fit(assmb_train)
val_fold_1 = rfModel.transform(assmb_val)
predictions_fold_1 = rfModel.transform(assmb_holdout)

In [0]:
display(predictions_fold_1)

YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,DEP_DEL15,DEP_DELAY_GROUP,DEP_TIME_BLK,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,CANCELLED,ACTUAL_ELAPSED_TIME,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,LATE_AIRCRAFT_DELAY,ORI_IATA,ORI_station_id,ORI_station_lat,ORI_station_lon,ORI_airport_lat,ORI_airport_lon,ORI_elevation,ORI_dist_airp_sta,DEST_IATA,DEST_station_id,DEST_station_lat,DEST_station_lon,DEST_airport_lat,DEST_airport_lon,DEST_elevation,DEST_dist_airp_sta,CRS_DEP_HRS,CRS_DEP_MINS,CRS_DEP_TIME_STR,CRS_DEP_DT_STR,CRS_DEP_DATETIME,iata_code,ORI_timezone,CRS_DEP_DATETIME_UTC,CRS_DEP_DATETIME_UTC_END,CRS_DEP_DATETIME_UTC_START,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,REPORT_TYPE,DEST_timezone,DEP_HRS,DEP_MINS,DEP_TIME_STR,DEP_DT_STR,DEP_DATETIME,ARR_DATETIME_ACTUAL_UTC,late_night,daytime,evening,region_name,new_england,mid_atlantic,south,midwest,southwest,west,pacific_islands,atlantic_islands,spring,summer,autumn,winter,dep_date,weekend_or_holiday,flightID,ID,previous_flight_delay_status,previous_flight_dep_time,time_between_departures_min,valid_dep_delay,prior_dep_delayed,previous_DEP_DELAY_NEW_value,previous_DEP_DELAY_NEW,previous_flight_arrdelay_status,previous_flight_arr_time,time_between_arrival_and_end_min,valid_arr_delay,prior_arr_delayed,previous_ARR_DELAY_NEW_value,previous_ARR_DELAY_NEW,prev_arrival_airport,plane_is_here,avg_carrier_delay_24hrs,flights_sch_Today_ORIGIN,flights_sch_Today_DEST,avg_ori_airport_delay_24hrs,year_quarter,quarter_enum,quarter_enum_prev,month_seq_index,wnd_2_wind_obs_type,wnd_3_wind_sp_rate,cig_0_height,vis_0_distance,slp_0_day_avg,tmp_0_air_temp,dew_0_point_temp,ma1_0_altimeter_setting_rate,ma1_2_station_pressure_rate,gd1_3_sky_cover_height,WND_3_wind_speed,CIG_0_sky_ceiling_height,VIS_0_visibility_dist,SLP_0_avg_station_press,TMP_0_air_temperature,DEW_0_dew_pt_temp,MA1_0_altimeter_set_rate,MA1_2_station_pres_rate,OC1_0_wind_gust_spd_rate_imp,GD1_3_cloud_height,AA1_1_liquid_precip,AA3_1_liquid_precip,gd1_0_sky_coverage,au2_4_extreme_wind_weather,mv1_0_sand_dust_near,mv1_0_thunder_rain_near,aw1_mw1_0_smoke_haze_dust,aw1_mw1_0_fog,aw1_mw1_0_rain_drizzle,aw1_mw1_0_freezing_rain_drizzle,aw1_mw1_0_snow,aw1_mw1_0_hail_or_ice,aw1_mw1_0_thunderstorm,aw1_mw1_0_tornado,SLP_0_avg_station_press_imp,WND_3_wind_speed_imp,CIG_0_sky_ceiling_height_imp,VIS_0_visibility_dist_imp,TMP_0_air_temperature_imp,DEW_0_dew_pt_temp_imp,MA1_0_altimeter_set_rate_imp,MA1_2_station_pres_rate_imp,GD1_3_cloud_height_imp,depDelayPageRank,arrDelayPageRank,UniqueID,hour,hour_UTC,avg_hourly_delay_24hr,OD_Pair,avg_OD_dep_del15_24hr,avg_OD_delay_min_24hr,avg_OD_num_flights_24hr,depDelayPageRank_ordinal,arrDelayPageRank_ordinal,recent_dep_time,recent_dep_dest,time_between_currentDepTime_recentDepTime,dep_flight_after_prediction,recent_dep_flight_coming_here,departed_for_current_aiport,1st_most_recent_dep_UTC,2nd_most_recent_dep_UTC,1st_most_recent_arr_UTC,2nd_most_recent_arr_UTC,3rd_most_recent_arr_UTC,col1_time_inb_flight_min,col2_time_inb_flight_min,col3_time_inb_flight_min,time_inb_flight_min,avg_time_inb_flights_carrier_24hr,avg_time_inb_flights_origin_24hr,avg_time_inb_flights_dest_24hr,avg_ori_DEP_DELAY_NEW_24hr,airline_carrier_del,airline_carrier_del_min,avg_carrier_delay_over15_lastQ,airline_carrier_del_ordinal,airline_carrier_del_min_ordinal,origin_avg_DEP_DEL15,origin_avg_DEP_DELAY_NEW,avg_origin_delay_over15_lastQ,origin_del_ordinal,origin_del_min_ordinal,OD_avg_DEP_DEL15,OD_avg_DEP_DELAY_NEW,avg_OD_delay_over15_lastQ,OD_del_ordinal,OD_del_min_ordinal,features,rawPrediction,probability,prediction
2019,1,1,1,2,2019-01-01,MQ,20398,N259NN,3574,11624,1162402,EYW,FL,12,Florida,11298,1129806,

In [0]:
feature_for_analysis = ['UniqueID','DEP_DEL15','probability','prediction','features']
RF_test_preds_fold_1 = predictions_fold_1.select(feature_for_analysis).withColumnRenamed('DEP_DEL15', 'label')

In [0]:
desired_neg_to_pos_ratio = 1.6

maxDepth=8
maxBins=64
minInstancesPerNode=10
numTrees=100
seed=12345

CV_name = 'fold_2'
train_data = df_imported.where((col('YEAR').cast('int') == 2016))
val_data = df_imported.where((col('YEAR').cast('int') == 2017))
holdout_data = df_imported.where((col('YEAR').cast('int') == 2019))
save_name = 'xgboost_test_preds_4_9_'+CV_name

new_trainDF = rebalanceDF(train_data, desired_neg_to_pos_ratio)
val = val_data
holdout = holdout_data

# Assemble features
assembler = VectorAssembler().setInputCols(feature_list_full).setOutputCol('features')
assmb_train = assembler.transform(new_trainDF)
assmb_val = assembler.transform(val)
assmb_holdout = assembler.transform(holdout)

# modeling
rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'DEP_DEL15',
                            maxDepth=maxDepth, maxBins=maxBins, minInstancesPerNode=minInstancesPerNode, numTrees=numTrees, seed=seed)
rfModel = rf.fit(assmb_train)
val_fold_2 = rfModel.transform(assmb_val)
predictions_fold_2 = rfModel.transform(assmb_holdout)

In [0]:
RF_test_preds_fold_2 = predictions_fold_2.select(feature_for_analysis).withColumnRenamed('DEP_DEL15', 'label')

In [0]:
desired_neg_to_pos_ratio = 1.6

maxDepth=8
maxBins=64
minInstancesPerNode=10
numTrees=100
seed=12345

CV_name = 'fold_3'
train_data = df_imported.where((col('YEAR').cast('int') == 2017))
val_data = df_imported.where((col('YEAR').cast('int') == 2018))
holdout_data = df_imported.where((col('YEAR').cast('int') == 2019))
save_name = 'xgboost_test_preds_4_9_'+CV_name

new_trainDF = rebalanceDF(train_data, desired_neg_to_pos_ratio)
val = val_data
holdout = holdout_data

# Assemble features
assembler = VectorAssembler().setInputCols(feature_list_full).setOutputCol('features')
assmb_train = assembler.transform(new_trainDF)
assmb_val = assembler.transform(val)
assmb_holdout = assembler.transform(holdout)

# modeling
rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'DEP_DEL15',
                            maxDepth=maxDepth, maxBins=maxBins, minInstancesPerNode=minInstancesPerNode, numTrees=numTrees, seed=seed)
rfModel = rf.fit(assmb_train)
val_fold_3 = rfModel.transform(assmb_val)
predictions_fold_3 = rfModel.transform(assmb_holdout)

In [0]:
RF_test_preds_fold_3 = predictions_fold_3.select(feature_for_analysis).withColumnRenamed('DEP_DEL15', 'label')

In [0]:
desired_neg_to_pos_ratio = 1.6

maxDepth=8
maxBins=64
minInstancesPerNode=10
numTrees=100
seed=12345

CV_name = 'fold_4'
train_data = df_imported.where(((col('YEAR').cast('int') == 2017) & (col('MONTH').cast('int') > 6)) | ((col('YEAR').cast('int') == 2018) & (col('MONTH').cast('int') < 7)))
val_data = df_imported.where(((col('YEAR').cast('int') == 2018) & (col('MONTH').cast('int') > 6)))
holdout_data = df_imported.where((col('YEAR').cast('int') == 2019))
save_name = 'xgboost_test_preds_4_9_'+CV_name

new_trainDF = rebalanceDF(train_data, desired_neg_to_pos_ratio)
val = val_data
holdout = holdout_data

# Assemble features
assembler = VectorAssembler().setInputCols(feature_list_full).setOutputCol('features')
assmb_train = assembler.transform(new_trainDF)
assmb_val = assembler.transform(val)
assmb_holdout = assembler.transform(holdout)

# modeling
rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'DEP_DEL15',
                            maxDepth=maxDepth, maxBins=maxBins, minInstancesPerNode=minInstancesPerNode, numTrees=numTrees, seed=seed)
rfModel = rf.fit(assmb_train)
val_fold_4 = rfModel.transform(assmb_val)
predictions_fold_4 = rfModel.transform(assmb_holdout)

In [0]:
RF_test_preds_fold_4 = predictions_fold_4.select(feature_for_analysis).withColumnRenamed('DEP_DEL15', 'label')

In [0]:
desired_neg_to_pos_ratio = 1.6

maxDepth=8
maxBins=64
minInstancesPerNode=10
numTrees=100
seed=12345

CV_name = 'fold_5'
train_data = df_imported.where(((col('YEAR').cast('int') == 2016) & (col('MONTH').cast('int') > 6)) | ((col('YEAR').cast('int') == 2017) & (col('MONTH').cast('int') < 7)))
val_data = df_imported.where(((col('YEAR').cast('int') == 2017) & (col('MONTH').cast('int') > 6)) | ((col('YEAR').cast('int') == 2018) & (col('MONTH').cast('int') < 7)))
holdout_data = df_imported.where((col('YEAR').cast('int') == 2019))
save_name = 'xgboost_test_preds_4_9_'+CV_name

new_trainDF = rebalanceDF(train_data, desired_neg_to_pos_ratio)
val = val_data
holdout = holdout_data

# Assemble features
assembler = VectorAssembler().setInputCols(feature_list_full).setOutputCol('features')
assmb_train = assembler.transform(new_trainDF)
assmb_val = assembler.transform(val)
assmb_holdout = assembler.transform(holdout)

# modeling
rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'DEP_DEL15',
                            maxDepth=maxDepth, maxBins=maxBins, minInstancesPerNode=minInstancesPerNode, numTrees=numTrees, seed=seed)
rfModel = rf.fit(assmb_train)
val_fold_5 = rfModel.transform(assmb_val)
predictions_fold_5 = rfModel.transform(assmb_holdout)

In [0]:
RF_test_preds_fold_5 = predictions_fold_5.select(feature_for_analysis).withColumnRenamed('DEP_DEL15', 'label')

In [0]:
feature_for_analysis = ['UniqueID','DEP_DEL15','probability','prediction','features']
RF_val_preds_fold_1 = val_fold_1.select(feature_for_analysis).withColumnRenamed('DEP_DEL15', 'label')
RF_val_preds_fold_2 = val_fold_2.select(feature_for_analysis).withColumnRenamed('DEP_DEL15', 'label')
RF_val_preds_fold_3 = val_fold_3.select(feature_for_analysis).withColumnRenamed('DEP_DEL15', 'label')
RF_val_preds_fold_4 = val_fold_4.select(feature_for_analysis).withColumnRenamed('DEP_DEL15', 'label')
RF_val_preds_fold_5 = val_fold_5.select(feature_for_analysis).withColumnRenamed('DEP_DEL15', 'label')

RF_test_preds_fold_1 = predictions_fold_1.select(feature_for_analysis).withColumnRenamed('DEP_DEL15', 'label')
RF_test_preds_fold_2 = predictions_fold_2.select(feature_for_analysis).withColumnRenamed('DEP_DEL15', 'label')
RF_test_preds_fold_3 = predictions_fold_3.select(feature_for_analysis).withColumnRenamed('DEP_DEL15', 'label')
RF_test_preds_fold_4 = predictions_fold_4.select(feature_for_analysis).withColumnRenamed('DEP_DEL15', 'label')
RF_test_preds_fold_5 = predictions_fold_5.select(feature_for_analysis).withColumnRenamed('DEP_DEL15', 'label')

In [0]:
RF_val_preds_fold_1.write.mode("overwrite").parquet(f"{blob_url}/RF_val_pred_table_cvgroup1_0410_xgbm")
RF_val_preds_fold_2.write.mode("overwrite").parquet(f"{blob_url}/RF_val_pred_table_cvgroup2_0410_xgbm")
RF_val_preds_fold_3.write.mode("overwrite").parquet(f"{blob_url}/RF_val_pred_table_cvgroup3_0410_xgbm")
RF_val_preds_fold_4.write.mode("overwrite").parquet(f"{blob_url}/RF_val_pred_table_cvgroup4_0410_xgbm")
RF_val_preds_fold_5.write.mode("overwrite").parquet(f"{blob_url}/RF_val_pred_table_cvgroup5_0410_xgbm")

RF_test_preds_fold_1.write.mode("overwrite").parquet(f"{blob_url}/RF_pred_table_cvgroup1_0410_xgbm")
RF_test_preds_fold_2.write.mode("overwrite").parquet(f"{blob_url}/RF_pred_table_cvgroup2_0410_xgbm")
RF_test_preds_fold_3.write.mode("overwrite").parquet(f"{blob_url}/RF_pred_table_cvgroup3_0410_xgbm")
RF_test_preds_fold_4.write.mode("overwrite").parquet(f"{blob_url}/RF_pred_table_cvgroup4_0410_xgbm")
RF_test_preds_fold_5.write.mode("overwrite").parquet(f"{blob_url}/RF_pred_table_cvgroup5_0410_xgbm")